In [17]:
import numpy as np
import json
import itertools

In [18]:
#Merge fidelity_list
def merge(f_list_all):
    f_list_merged = {}
    for f_list in f_list_all:
        for plabel, f_p in f_list.items():
            if plabel not in f_list_merged:
                f_list_merged[plabel] = f_p
            else:
                for L, f_L in f_p.items():
                    if L not in f_list_merged[plabel]:
                        f_list_merged[plabel][L] = f_L
                    else:
                        f_list_merged[plabel][L] += f_L
    return f_list_merged

In [19]:
#std or cor

fname_list = ['batch6/simulate_cb_single.txt']
circuit_config = 'cor'

# if circuit_config == 'std':
#     fname_list = ["batch3_std/simulate_cb_single.txt","batch4_std/simulate_cb_single.txt","batch5_std/simulate_cb_single.txt"]
# else:
#     fname_list = ["batch1/simulate_cb_single.txt","batch2/simulate_cb_single.txt","batch6/simulate_cb_single.txt"]

# fname_list = 'batch1/simulate_cb_single.txt'
# circuit_config = 'cor'

fid_all = []
for fname in fname_list:
    with open(fname,"r") as f:
        fid_all += json.load(f)
fid_merged = merge(fid_all)

In [20]:
import CB_process
n = 2
# std
if circuit_config == 'std':
    unlearnable_pauli_parity_sample_list = [('ZZ',0),('ZI',1),('XX',0),('IX',1),('YZ',0),('YI',1),('XY',0),('IY',1),('ZY',0),('YX',1),('ZX',0),('YY',1)]
    learnable_pauli_parity_sample_list = [('IZ',0),('XI',0),('XZ',0)]
else:
    unlearnable_pauli_parity_sample_list = [('ZZ',0),('YI',1),('XX',0),('IY',1),('YZ',0),('ZI',1),('XY',0),('IX',1)]
    learnable_pauli_parity_sample_list = [('ZY',0),('YX',0),('ZX',0),('YY',0),('IZ',0),('XI',0),('XZ',0)]

N_un = len(unlearnable_pauli_parity_sample_list)

pauli_parity_sample_list = unlearnable_pauli_parity_sample_list + learnable_pauli_parity_sample_list




In [21]:
fidelity_list = {} 
stdev_list = {}
intercept_list = {}
intercept_std_list = {}

for label in fid_merged.keys():
    Lrange = list(fid_merged[label].keys())
    #Lrange = [int(L) for L in fid_merged[label].keys()]
    parity = int(Lrange[0])%2

    if (label,parity) in fidelity_list:
        print("What happen?")
        continue
    elif(label != 'I'*n):
        alpha, alpha_err, intercept, intercept_err = CB_process.fit_CB_2(Lrange, fid_merged[label])
        # print("CB for %dth Pauli:" % k, alpha, alpha_err)
        fidelity_list[(label,parity)] = alpha
        stdev_list[(label,parity)] = alpha_err
        intercept_list[(label,parity)] = intercept
        intercept_std_list[(label,parity)] = intercept_err

# add the trivial one
fidelity_list[('I'*n,0)] = 1.0
stdev_list[('I'*n,0)] = 0.0
intercept_list[('I'*n,0)] = 1.0
intercept_std_list[('I'*n,0)] = 0.0

In [22]:
print("Label / Pauli infidelity / Std(fidelity) / Intercept / Std(intercept)")
for pauli_label in pauli_parity_sample_list:
    # print(pauli_label, 1-fidelity_list[pauli_label], stdev_list[pauli_label], intercept_list[pauli_label], intercept_std_list[pauli_label])
    print(str(pauli_label)+" %.5f %.5f %.5f %.5f"%(1-fidelity_list[pauli_label], stdev_list[pauli_label], intercept_list[pauli_label], intercept_std_list[pauli_label]))

Label / Pauli infidelity / Std(fidelity) / Intercept / Std(intercept)
('ZZ', 0) 0.01371 0.00059 0.88134 0.00814
('YI', 1) 0.01212 0.00042 0.89474 0.00496
('XX', 0) 0.01225 0.00056 0.88592 0.00610
('IY', 1) 0.01327 0.00045 0.91201 0.00494
('YZ', 0) 0.01572 0.00065 0.90240 0.00712
('ZI', 1) 0.01393 0.00045 0.91646 0.00524
('XY', 0) 0.01534 0.00053 0.90513 0.00608
('IX', 1) 0.01476 0.00048 0.90397 0.00543
('ZY', 0) 0.01512 0.00038 0.88759 0.00636
('YX', 0) 0.01729 0.00061 0.89722 0.00690
('ZX', 0) 0.01962 0.00035 0.89921 0.00597
('YY', 0) 0.01838 0.00071 0.88735 0.00730
('IZ', 0) 0.00832 0.00020 0.93522 0.00394
('XI', 0) 0.01483 0.00058 0.96055 0.00364
('XZ', 0) 0.02620 0.00080 0.89282 0.00654


In [23]:
noise_after_the_gate = True

dec_fidelity_list = {}
for k in range(0,N_un,2):
    pp1 = pauli_parity_sample_list[k]
    pp2 = pauli_parity_sample_list[k+1]
    f = (fidelity_list[pp1]+fidelity_list[pp2])/2
    r = (intercept_list[pp2]/intercept_list[pp1])
    if noise_after_the_gate is True:
        lambda1 = f * r
        lambda2 = f / r
    else:
        lambda1 = f / r
        lambda2 = f * r

    dec_fidelity_list[pp1[0]] = lambda1
    dec_fidelity_list[pp2[0]] = lambda2

for k in range(N_un,15):
    pauli_label = pauli_parity_sample_list[k]
    dec_fidelity_list[pauli_label[0]] = fidelity_list[pauli_label]
# print(dec_fidelity_list)

dec_fidelity_list['II'] = 1.0


full_pauli_list = [''.join(s) for s in itertools.product(['I','X','Y','Z'], repeat = n)]

print("Label / Pauli infidelity (decoupled)")
for pauli_label in pauli_parity_sample_list:
# for pauli_label in full_pauli_list:
    # print(pauli_label, 1-fidelity_list[pauli_label], stdev_list[pauli_label], intercept_list[pauli_label], intercept_std_list[pauli_label])
    print(str(pauli_label[0])+" %.5f"%(1-dec_fidelity_list[pauli_label[0]]))
    # print(str(pauli_label)+" %.5f"%(1-dec_fidelity_list[pauli_label]))



Label / Pauli infidelity (decoupled)
ZZ -0.00209
YI 0.02768
XX -0.01632
IY 0.04100
YZ -0.00052
ZI 0.02994
XY 0.01632
IX 0.01378
ZY 0.01512
YX 0.01729
ZX 0.01962
YY 0.01838
IZ 0.00832
XI 0.01483
XZ 0.02620


In [24]:
1-np.average(list(fidelity_list.values()))

0.014428313422270245

In [25]:
fid_merged

{'ZZ': {'2': [0.92,
   0.936,
   0.93,
   0.928,
   0.864,
   0.914,
   0.868,
   0.888,
   0.762,
   0.776,
   0.918,
   0.85,
   0.836,
   0.864,
   0.864,
   0.83,
   0.812,
   0.89,
   0.89,
   0.884,
   0.86,
   0.846,
   0.782,
   0.808,
   0.93,
   0.82,
   0.848,
   0.788,
   0.95,
   0.816],
  '6': [0.81,
   0.786,
   0.87,
   0.818,
   0.906,
   0.832,
   0.768,
   0.864,
   0.818,
   0.646,
   0.766,
   0.858,
   0.734,
   0.798,
   0.766,
   0.886,
   0.818,
   0.722,
   0.776,
   0.802,
   0.804,
   0.828,
   0.772,
   0.798,
   0.904,
   0.878,
   0.836,
   0.856,
   0.842,
   0.796],
  '10': [0.796,
   0.692,
   0.73,
   0.744,
   0.542,
   0.854,
   0.69,
   0.73,
   0.83,
   0.832,
   0.794,
   0.74,
   0.782,
   0.764,
   0.784,
   0.8,
   0.862,
   0.736,
   0.764,
   0.812,
   0.65,
   0.77,
   0.82,
   0.768,
   0.754,
   0.48,
   0.872,
   0.772,
   0.722,
   0.814],
  '14': [0.698,
   0.524,
   0.724,
   0.742,
   0.742,
   0.694,
   0.772,
   0.71,
   0.412,
   